In [426]:
import warnings 
warnings.filterwarnings("ignore")
from sklearn.linear_model import ElasticNet

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder , PolynomialFeatures 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
train = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/train.csv')
test = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/test.csv')
sample_submission = pd.read_csv('/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission.csv')

In [427]:
data = pd.concat([train , test] , axis= 0)
test_id = test["Id"]
data.drop(["Id"] ,axis=1, inplace=True)
data["Age"] = data["YrSold"] - data["YearBuilt"]
# data["Agere"] = data["YrSold"] - data["YearRemodAdd"]

In [428]:
str_cols = data.select_dtypes("O").columns
num_cols = list(set(data.columns) - set(str_cols))
no_cols = ["MiscFeature" , "Fence" , "GarageCond" , "GarageQual" , "GarageFinish" , "FireplaceQu" , "BsmtFinType2" , "BsmtExposure", "BsmtCond" , "BsmtQual" , "Alley"]
for col in test.columns:
    if col != "SalePrice":
        if col in no_cols:
            data[col]=data[col].fillna("no")
        elif col in str_cols:
            data[col]=data[col].fillna(data[col].mode()[0])
        elif col in num_cols:
            data[col]=data[col].fillna(data[col].median())

data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Age
0,60,RL,65.0,8450,Pave,no,Reg,Lvl,AllPub,Inside,...,Ex,no,no,0,2,2008,WD,Normal,208500.0,5
1,20,RL,80.0,9600,Pave,no,Reg,Lvl,AllPub,FR2,...,Ex,no,no,0,5,2007,WD,Normal,181500.0,31
2,60,RL,68.0,11250,Pave,no,IR1,Lvl,AllPub,Inside,...,Ex,no,no,0,9,2008,WD,Normal,223500.0,7
3,70,RL,60.0,9550,Pave,no,IR1,Lvl,AllPub,Corner,...,Ex,no,no,0,2,2006,WD,Abnorml,140000.0,91
4,60,RL,84.0,14260,Pave,no,IR1,Lvl,AllPub,FR2,...,Ex,no,no,0,12,2008,WD,Normal,250000.0,8


In [429]:
ohe  = OneHotEncoder(drop = "first")
ohe.fit(data[str_cols])
ohe_data=ohe.transform(data[str_cols]).toarray()
ohe_df = pd.DataFrame(ohe_data , columns = ohe.get_feature_names_out())

In [430]:
if list(str_cols) in list(data.columns):
    data.drop(str_cols , inplace=True ,axis = 1)
data.index=range(len(data))

ohe_df.index=range(len(ohe_df))

In [431]:
data=pd.concat([data, ohe_df], axis=1)

In [432]:
a = data.drop(["SalePrice"] , axis=1)
# ct = ColumnTransformer(transformers=[( "poly" ,PolynomialFeatures(degree=2), a.select_dtypes(include=['number']).columns)], remainder='passthrough')
# ct.fit_transform(data)
data.select_dtypes("number")

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2915,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2916,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2917,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [433]:
# cte = ColumnTransformer(transformers=[( "poly" ,PolynomialFeatures(degree=2), pd.Series((list(data.select_dtypes(include=['number']).columns).remove("SalePrice"))).astype("O"))], remainder='passthrough')
# cte.fit_transform(data)
# data.select_dtypes("number")

In [434]:
from sklearn.compose import ColumnTransformer

train=data[data['SalePrice'].notnull()]
test=data[data['SalePrice'].isnull()]
test.drop(['SalePrice'] , inplace=True , axis = 1)

X=train.drop('SalePrice', axis=1)
Y=train['SalePrice']

# # One-hot encode categorical vari
# # ables
ct = ColumnTransformer([('encoder', OneHotEncoder(), X.select_dtypes(include=['object']).columns)], remainder='passthrough')
X = ct.fit_transform(X)

X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size=0.3 , random_state=42)

In [435]:
def Poly_F(x , y , i , test ):
    pf = PolynomialFeatures(degree=i)
    pf.fit(x)
    x_poly = pd.DataFrame(pf.transform(x) , columns = pf.get_feature_names_out())
    pfe = PolynomialFeatures(degree=i)
    pfe.fit(test)
    test = pd.DataFrame(pfe.transform(test) , columns = pfe.get_feature_names_out())

    X_train , X_test , Y_train , Y_test = train_test_split(x_poly , y , test_size=0.3 , random_state=42)
    lr = LinearRegression()
    lr.fit(X_train , Y_train)
    # rmse_t = np.sqrt(mean_squared_error(Y_train , lr.predict(X_train)))
    # rmse= np.sqrt(mean_squared_error(Y_test , lr.predict(X_test)))
    return lr.predict(test)

In [436]:
def enet(x , y , i , j):
    enet = ElasticNet(alpha= i  , l1_ratio=j)
    X_train , X_test , Y_train , Y_test = train_test_split(x , y , test_size=0.3 , random_state=42)
    enet.fit(X_train , Y_train)
    rmse_t = np.sqrt(mean_squared_error(Y_train , enet.predict(X_train)))
    rmse= np.sqrt(mean_squared_error(Y_test , enet.predict(X_test)))
    return rmse_t , rmse , i , j

In [437]:
best_enet = {}
best_enet = dict(best_enet)
best_enety = {}
best_enety = dict(best_enety)
# for i in [1,20,80]:
#     for j in [1,20,80]:
#         a , b , c , d= enet(X ,Y ,i/100 , j/100)
#         best_enet[f" i {c} , j {d}"] = b
#         best_enety[f" i {c} , j {d}"] = a


KeyboardInterrupt: 

In [ ]:
print(print(min(best_enet, key=best_enet.get)))
print(print(min(best_enety, key=best_enety.get)))

 i 0.01 , j 0.8
None
 i 0.01 , j 0.8
None


In [ ]:
#KNN - 0.39
#knn = KNeighborsClassifier()
#knn.fit(X_train , Y_train)

In [ ]:
#Poly degree 2
lr = LinearRegression()
lr.fit(X , Y)


LinearRegression()

In [ ]:
from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=0.1, l1_ratio=0.7)
enet.fit(X , Y)

ElasticNet(alpha=0.01, l1_ratio=0.8)

In [ ]:


filename = "/Users/hajiaga/Documents/house-prices-advanced-regression-techniques-2/sample_submission2.csv"
import os

test_ohe = ct.transform(test)

test_pred = enet.predict(test_ohe)

sample_submission['SalePrice'] = pd.Series(test_pred)

if os.path.exists(filename):
    os.remove(filename)
sample_submission.to_csv(filename, index=False) 
sample_submission.head()

,Id,SalePrice
0,1461,105283.087437
1,1462,145530.932080
2,1463,177076.219438
3,1464,189441.748577
4,1465,210815.814608
